### 1. 数据处理

In [1]:
# from modelscope import snapshot_download

# model_dir = snapshot_download(
#     'qwen/Qwen2.5-7B-Instruct', 
#     cache_dir='./qwen_model', 
#     revision='master'
# )

# print(f"模型已下载到: {model_dir}")

In [2]:
import json
from rich import print

import os
# 切换到国内镜像源，秒级下载
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

def parse_BIO_file(file_path):
    """
    从 BIO 格式的数据中提取句子和对应的标签序列。
    """
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        chars = []
        tags = []
        for line in f:
            line = line.strip()
            # 如果是空行，说明一句话结束了
            if not line:
                data.append((chars, tags))
                chars = []
                tags = []
                continue
            
            # 分割字符和标签
            if len(line) == 1:
                # print(line)
                char = ' '
                tag = 'O'
            else :
                char, tag = line.split()
            chars.append(char)
            tags.append(tag)
                
        # 防止文件最后没有空行导致漏掉最后一句
        if chars: 
            data.append((chars, tags))
    return data

def extract_entities(chars, tags):
    """
    提取实体，识别B-和I-，拼成完整的实体
    """
    entities = {}
    i = 0
    while i < len(tags):
        tag = tags[i]
        if tag.startswith("B-"):
            entity_type = tag[2:] # 截取类别名 (去掉 B-)
            if entity_type not in entities: entities[entity_type] = []
            entity_content = chars[i]
            j = i + 1
            # 继续向后找同类型的 I- 标签
            while j < len(tags) and tags[j] == f"I-{entity_type}":
                entity_content += chars[j]
                j += 1
            entities[entity_type].append(entity_content)
            i = j
        else : i += 1
    return entities

def convert_to_json(input_file, output_file):
    """
    处理 BIO 数据，得到模型训练所需的 JSON 对话格式
    """
    raw_data = parse_BIO_file(input_file)
    system_prompt = (
        "你是一个中医药领域的命名实体识别专家。请从文本中提取以下类别的实体并以JSON格式输出："
        "['中医治则', '中医治疗', '中医证候', '中医诊断', '中药', '临床表现', '其他治疗', '方剂', '西医治疗', '西医诊断']。"
        "如果文本中不包含任何上述实体，请输出：{}。"
        )
    with open(output_file, 'w', encoding='utf-8') as f:
        for chars, tags in raw_data:
            
            text = "".join(chars)
            # print(text)
            entity_dict = extract_entities(chars, tags)
            
            # print(entity_dict)
            if not entity_dict:
                print("wrong")
                assistant_content = json.dumps({"result": "未找到相关实体"}, ensure_ascii=False)
            else :
                assistant_content = json.dumps(entity_dict, ensure_ascii=False)
            
            message = {
                "messages": [
                    {
                        "role": "system",
                        "content": system_prompt
                    },
                    {
                        "role": "user",
                        "content": text
                    },
                    {
                        "role": "assistant",
                        "content": assistant_content
                    }
                ]
            }
            f.write(json.dumps(message, ensure_ascii=False) + "\n")
    print(f"转换完成：{output_file}")

In [3]:
convert_to_json('./data/medical.dev', './data/medical_dev.jsonl')
convert_to_json('./data/medical.test', './data/medical_test.jsonl')
convert_to_json('./data/medical.train', './data/medical_train.jsonl')

转换完成：./data/medical_dev.jsonl

转换完成：./data/medical_test.jsonl

转换完成：./data/medical_train.jsonl

### 2. 加载 Base Model

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# model_id = "Qwen/Qwen2.5-7B-Instruct"
model_id = "/hy-tmp/qwen_model/qwen/Qwen2.5-7B-Instruct"
print(f"加载 Tokenizer: {model_id} ...")
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True, trust_remote_code=True)
# tokenizer.padding_side = 'left'
print(f"加载 Tokenizer: Done! ")

print("加载模型...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16, 
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)
print("加载模型: Done! ")

print("测试模型：")
prompt = "简单介绍下你自己"
messages = [
    {"role": "system", "content": "你是一个人工智能助手。"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors='pt').to(model.device)
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=50
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f"用户：{prompt}")
print(f"模型：{response}")

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


加载 Tokenizer: /hy-tmp/qwen_model/qwen/Qwen2.5-7B-Instruct ...

加载 Tokenizer: Done!

加载模型...

Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.25s/it]


加载模型: Done!

测试模型：

用户：简单介绍下你自己

模型：我叫通义千问，是由阿里云开发的大型语言模型。我的主要功能是生成各种文本内容，比如撰写文章、编写代码、创作诗歌
，甚至回答问题和解决问题等。我旨在提供准确、有用的信息，

### 3. 配置 LoRA

In [5]:
from peft import LoraConfig, TaskType, get_peft_model

target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

# LoRA 配置
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=target_modules
)

model = get_peft_model(model, peft_config)

model.print_trainable_parameters()

trainable params: 40,370,176 || all params: 7,655,986,688 || trainable%: 0.5273


### 4. 配置训练参数

In [6]:
import numpy as np
import json
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset

def extract_entities_from_json(json_str):
    """
    从模型生成的字符串中提取实体
    """
    entities = set()
    try:
        # 切割 assistant 防止 System Prompt 干扰
        if "assistant" in json_str:
            json_str = json_str.split("assistant")[-1]

        json_str = json_str.strip()
        start = json_str.find('{')
        end = json_str.rfind('}') + 1
        if start != -1 and end != 0:
            data = json.loads(json_str[start:end])
            for label, items in data.items():
                # 如果是 "result" 直接跳过
                # if label == "result":
                #     continue
                if isinstance(items, str):
                    items = [items]
                if isinstance(items, list):
                    for item in items:
                        if isinstance(item, str) and len(item.strip()) > 0:
                            entities.add((label, item.strip()))
    except Exception as e:
        print("\n" + "="*20 + " JSON 解析失败 " + "="*20)
        print(f"错误类型: {type(e).__name__}")
        print(f"错误信息: {e}")
        print(f"模型输出:\n{json_str}")
        print("="*55 + "\n")
        pass
    return entities

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # 将 preds 中的 -100 替换为 pad_token_id
    #如果不做这一步，tokenizer.batch_decode 解码 -100 时就会报 OverflowError
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    
    # 将 labels 中的 -100 替换为 pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    

    
    total_tp, total_fp, total_fn = 0, 0, 0
    for pred_str, label_str in zip(decoded_preds, decoded_labels):
        pred_entities = extract_entities_from_json(pred_str)
        true_entities = extract_entities_from_json(label_str)
        print(f"【真实输出】: {label_str}")
        print(f"【模型输出】: {pred_str}")
        print(f"【真实标签】: {true_entities}")
        print(f"【模型预测】: {pred_entities}")
        tp = len(pred_entities & true_entities)
        fp = len(pred_entities - true_entities)
        fn = len(true_entities - pred_entities)
        
        total_tp += tp
        total_fp += fp
        total_fn += fn
    
    precision = total_tp / (total_tp + total_fp + 1e-5)
    recall = total_tp / (total_tp + total_fn + 1e-5)
    f1 = 2 * precision * recall / (precision + recall + 1e-5)
    
    print(f"\n[Eval] Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")
    
    return {"precision": precision, "recall": recall, "f1": f1}



from transformers import TrainerCallback
import swanlab
import random

class F1ScoreCallback(TrainerCallback):
    def __init__(self, tokenizer, eval_data, log_step=50):
        self.tokenizer = tokenizer
        self.eval_data = eval_data 
        self.log_step = log_step
        
    def on_evaluate(self, args, state, control, model, **kwargs):
        """
        每次 Trainer 做完评估 (计算完 Loss) 后，会自动触发这个函数
        """
        print("\n [Callback] 正在进行 F1 生成式评估...")
        model.eval()
        total_tp, total_fp, total_fn = 0, 0, 0
        
        # 如果验证集很大，建议 sample 一下，否则训练会很慢
        eval_list = random.sample(list(self.eval_data), 10)
        
        for item in eval_list:
            # 1. 构建 Prompt
            prompt = self.tokenizer.apply_chat_template(
                item["messages"][:-1], 
                add_generation_prompt=True, 
                tokenize=False
            )
            inputs = self.tokenizer(prompt, return_tensors="pt").to(model.device)
            
            # 2. 生成
            with torch.no_grad():
                outputs = model.generate(
                    **inputs, 
                    max_new_tokens=512,
                    pad_token_id=self.tokenizer.pad_token_id
                )
            
            # 3. 计算 F1
            pred_str = self.tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
            label_str = item["messages"][-1]["content"]
            
            pred_entities = extract_entities_from_json(pred_str)
            true_entities = extract_entities_from_json(label_str)
            
            total_tp += len(pred_entities & true_entities)
            total_fp += len(pred_entities - true_entities)
            total_fn += len(true_entities - pred_entities)

        # 4. 计算指标
        precision = total_tp / (total_tp + total_fp + 1e-5)
        recall = total_tp / (total_tp + total_fn + 1e-5)
        f1 = 2 * precision * recall / (precision + recall + 1e-5)
        
        print(f"评估结果 - F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")
        swanlab.log({"Validation/F1": f1, "Validation/Precision": precision, "Validation/Recall": recall}, step=state.global_step)


In [7]:
def process_func(example):
    """
    处理输入数据，将输入转为 Input IDs
    """
    MAX_LENGTH = 512
    input_ids, attention_mask, labels = [], [], []
    # 使用 tokenizer 的 chat template
    instruction = tokenizer.apply_chat_template(
        example["messages"][:-1], # 取 System + User
        add_generation_prompt=True,
        tokenize=False
    )
    # print(instruction)
    response = example["messages"][-1]["content"] # 取 Assistant 回复
    # print(response)
    instruction_ids = tokenizer(instruction, add_special_tokens=False).input_ids
    response_ids = tokenizer(response, add_special_tokens=False).input_ids + [tokenizer.eos_token_id]
    
    input_ids = instruction_ids + response_ids
    attention_mask = [1] * len(input_ids)
    # # 构造 Labels：User 部分设为 -100 (不计算 Loss)，Assistant 部分保留
    labels = [-100] * len(instruction_ids) + response_ids
    
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

def load_jsonl(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]


train_data = load_jsonl("./data/medical_train.jsonl")
eval_data = load_jsonl("./data/medical_dev.jsonl")
test_data = load_jsonl("./data/medical_test.jsonl")

train_ds = Dataset.from_list(train_data)
eval_ds = Dataset.from_list(eval_data)
test_ds = Dataset.from_list(test_data)

train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names)
eval_dataset = eval_ds.map(process_func, remove_columns=eval_ds.column_names)
test_dataset = test_ds.map(process_func, remove_columns=test_ds.column_names)
print(train_dataset)
print(eval_dataset)
print(test_dataset)


f1_callback = F1ScoreCallback(tokenizer=tokenizer, eval_data=eval_ds)

Map: 100%|██████████| 658/658 [00:00<00:00, 1910.94 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5259
})

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 657
})

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 658
})

In [8]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from swanlab.integration.transformers import SwanLabCallback
from transformers import EarlyStoppingCallback

# ---------------------------------------------------------------------
# 1. 配置 SwanLab 可视化
# ---------------------------------------------------------------------
swanlab_callback = SwanLabCallback(
    project="Qwen2.5-NER-Finetune",
    experiment_name="Qwen2.5-7B-LoRA-Run1",
    description="使用 QLoRA 微调 Qwen2.5-7B 进行中医药 NER，监控 F1 变化",
    config={
        "model": "Qwen/Qwen2.5-7B-Instruct",
        "learning_rate": 1e-4,
        "lora_rank": 16,
    }
)

print("SwanLab 配置完成")

# ---------------------------------------------------------------------
# 2. 设置训练参数 
# ---------------------------------------------------------------------
args = Seq2SeqTrainingArguments(
    output_dir="./output/Qwen2.5-NER", # 模型输出路径
    
    # --- 核心训练参数 ---
    per_device_train_batch_size=4,     
    gradient_accumulation_steps=8,     # 累积 8 步更新一次，变相增大 batch size
    learning_rate=1e-4,                # LoRA 常用学习率
    num_train_epochs=5,                
    lr_scheduler_type="cosine",        # 余弦退火
    warmup_ratio=0.1,                  # 预热 10% 的步数
    fp16=True,                         # 开启混合精度，省显存提速
    
    # --- 评估与保存策略 ---
    eval_strategy="steps",       # 按步数进行评估
    eval_steps=50,                     # 每训练 50 步，验证一次 F1
    save_strategy="steps",             # 按步数保存 Checkpoint
    save_steps=50,                     
    save_total_limit=1,                
    load_best_model_at_end=True,       
    metric_for_best_model="loss",        
    greater_is_better=False,
    predict_with_generate=False,
    
    # --- 日志 ---
    logging_steps=10,                  # 每 10 步打印一次 Loss
    report_to="none",                  # 禁用 wandb，完全交给 SwanLab 接管
)

# ---------------------------------------------------------------------
# 3. 初始化 Seq2SeqTrainer
# ---------------------------------------------------------------------
trainer = Seq2SeqTrainer(
    model=model,                       
    args=args,                        
    train_dataset=train_dataset,       
    eval_dataset=eval_dataset,         
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    processing_class=tokenizer,
    # compute_metrics=compute_metrics,
    callbacks=[
        swanlab_callback,
        f1_callback,
        EarlyStoppingCallback(early_stopping_patience=3)
    ],      
)

# ---------------------------------------------------------------------
# 4. 开始训练 
# ---------------------------------------------------------------------
print("开始训练...")
trainer.train()

# ---------------------------------------------------------------------
# 5. 保存最终模型
# ---------------------------------------------------------------------
final_save_path = "./output/final_model"
trainer.save_model(final_save_path)
print(f"训练完成，最佳模型已加载并保存至 {final_save_path}")

SwanLab 配置完成

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


开始训练...

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


/usr/local/lib/python3.11/dist-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

[Callback] 正在进行 F1 生成式评估...

评估结果 - F1: 0.6471 | Precision: 0.5789 | Recall: 0.7333

[Callback] 正在进行 F1 生成式评估...

评估结果 - F1: 0.8000 | Precision: 0.8333 | Recall: 0.7692

训练完成，最佳模型已加载并保存至 ./output/final_model

In [9]:
# test_text = "患者长期胃痛，喜温喜按，舌淡苔白，脉沉细。诊断为脾胃虚寒型胃痛，建议用黄芪建中汤治疗。"


# prompt = f"system\n你是一个中医药领域的命名实体识别专家。请从文本中提取以下类别的实体并以JSON格式输出：['中医治则', '中医治疗', '中医证候', '中医诊断', '中药', '临床表现', '其他治疗', '方剂', '西医治疗', '西医诊断']。如果文本中不包含任何上述实体，请输出：{{}}。\nuser\n{test_text}\nassistant"


# inputs = tokenizer(prompt, return_tensors="pt").to(model.device)


# output = model.generate(
#     **inputs, 
#     max_new_tokens=512,
#     use_cache=True,
#     pad_token_id=tokenizer.pad_token_id
# )


# response = tokenizer.decode(output[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True)

# print("======== 真实测试结果 ========")
# print(response)

In [11]:
from tqdm import tqdm
import torch
from collections import defaultdict

def run_manual_eval(model, tokenizer, test_data):
    """
    计算整体及各分类的 F1 Score
    """
    model.eval()
    global_tp, global_fp, global_fn = 0, 0, 0
    category_stats = defaultdict(lambda: {"tp": 0, "fp": 0, "fn": 0})
    # print(f"正在评估 {len(test_data)} 条数据...")
    
    # 遍历验证集
    for item in tqdm(test_data):
        # 1. 准备 Prompt (只给 System + User)
        prompt = tokenizer.apply_chat_template(
            item["messages"][:-1], 
            add_generation_prompt=True, 
            tokenize=False
        )
        
        # 2. 编码输入
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        # 3. 模型生成 
        with torch.no_grad():
            outputs = model.generate(
                **inputs, 
                max_new_tokens=512,
                use_cache=True,
                pad_token_id=tokenizer.pad_token_id
            )
        
        # 4. 解码预测结果
        # 只截取新生成的 tokens
        generated_ids = outputs[0][inputs.input_ids.shape[1]:]
        pred_str = tokenizer.decode(generated_ids, skip_special_tokens=True)
        
        # 5. 获取真实标签
        label_str = item["messages"][-1]["content"]
        
        # 6. 提取实体并计算
        pred_entities = extract_entities_from_json(pred_str)
        true_entities = extract_entities_from_json(label_str)
        
        tp_set = pred_entities & true_entities 
        fp_set = pred_entities - true_entities 
        fn_set = true_entities - pred_entities 
        
        global_tp += len(tp_set)
        global_fp += len(fp_set)
        global_fn += len(fn_set)

        for cat, name in tp_set:
            category_stats[cat]["tp"] += 1
        for cat, name in fp_set:
            category_stats[cat]["fp"] += 1
        for cat, name in fn_set:
            category_stats[cat]["fn"] += 1

    print("\n" + "="*20 + " 整体评估结果 " + "="*20)
    g_precision = global_tp / (global_tp + global_fp + 1e-5)
    g_recall = global_tp / (global_tp + global_fn + 1e-5)
    g_f1 = 2 * g_precision * g_recall / (g_precision + g_recall + 1e-5)
    print(f"Overall Precision : {g_precision:.4f}")
    print(f"Overall Recall    : {g_recall:.4f}")
    print(f"Overall F1 Score  : {g_f1:.4f}")
    
    print("\n" + "="*20 + " 各类别评估结果 " + "="*20)
    results_list = []
    for cat, stats in category_stats.items():
        tp = stats["tp"]
        fp = stats["fp"]
        fn = stats["fn"]
        
        p = tp / (tp + fp + 1e-5)
        r = tp / (tp + fn + 1e-5)
        f1 = 2 * p * r / (p + r + 1e-5)
        
        results_list.append({
            "Category": cat,
            "Precision": p,
            "Recall": r,
            "F1": f1,
        })

    results_list.sort(key=lambda x: x["F1"], reverse=True)

    print(f"{'Category':<15} | {'F1':<8} | {'Precision':<10} | {'Recall':<10}")
    print("-" * 65)
    for row in results_list:
        print(f"{row['Category']:<15} | {row['F1']:.4f}   | {row['Precision']:.4f}     | {row['Recall']:.4f}")
    print("-" * 65)

run_manual_eval(model, tokenizer, test_data)

正在评估 658 条数据...

100%|██████████| 658/658 [20:48<00:00,  1.90s/it]


==================== 整体评估结果 ====================

Overall Precision : 0.7289

Overall Recall    : 0.7891

...

方剂              | 0.6330   | 0.6389     | 0.6273

中医治则            | 0.6105   | 0.5686     | 0.6591

中医诊断            | 0.6046   | 0.6667     | 0.5532

其他治疗            | 0.5882   | 1.0000     | 0.4167

-----------------------------------------------------------------